## A Simple Pair Trading Strategy


Builds a simple pair trading strategy to show an example of a strategy that relies on relationships between multiple contracts.

1.  We will buy AAPL and sell NVDA when the price ratio AAPL / NVDA is more than 1 standard deviation lower than its 1 day simple moving average.  
2.  We will buy PEP and sell KO when the price ratio AAPL / NVDA is more than 1 standard deviation higher than its 1 day simple moving average.
3.  We will exit when the price ratio is less than +/- 0.5 standard deviations away from its simple moving average
4.  We will size the trades in 1 and 2 by allocating 10% of our capital to each trade.

In [6]:
%%checkall
import pandas as pd
import pyqstrat as pq

_logger = pq.get_child_logger(__name__)


if __name__ == '__main__':

    pq.set_defaults()  # Set some display defaults to make dataframes and plots easier to look at

    aapl = pd.read_csv('./support/AAPL.csv.gz', parse_dates=['timestamp'])[['timestamp', 'c']]
    nvda = pd.read_csv('./support/NVDA.csv.gz', parse_dates=['timestamp'])[['timestamp', 'c']]
    prices = pd.merge(aapl, nvda, on=['timestamp'], how='outer', suffixes=['_aapl', '_nvda'])
    prices['ratio'] = prices.c_aapl / prices.c_nvda
    
    r = pd.Series(prices.ratio).rolling(window=int(60 * 6.5))  # 1 day * 6.5 trading hours / day * 60 minutes / hour
    mean, std = r.mean(), r.std(ddof=0)
    zscore = (prices.ratio - mean) / std
    prices['zscore'] = zscore

    strat_builder = pq.StrategyBuilder(prices)
    strat_builder.add_contract('AAPL')
    strat_builder.add_contract('NVDA')

    price_dict = {'AAPL': {aapl.timestamp.values[i].astype('M8[m]'): aapl.c.values[i] for i in range(len(aapl))},
                  'NVDA': {nvda.timestamp.values[i].astype('M8[m]'): nvda.c.values[i] for i in range(len(nvda))}}
    price_func = pq.PriceFuncDict(price_dict)
    strat_builder.set_price_function(price_func)

    prices['enter_long'] = (prices.zscore > 1)
    prices['enter_short'] = (prices.zscore < -1)
    prices['exit_long'] = ((prices.zscore > 0.5) * (prices.zscore < 1))
    prices['exit_short'] = ((prices.zscore > 0.5) * (prices.zscore < 1))
    
    def get_direction(cg: pq.ContractGroup, long: bool) -> int:
        ret = 1
        if not long: ret *= -1
        if cg.name == 'NVDA': ret *= -1
        return ret
        
    long_entry_rule = pq.PercentOfEquityTradingRule('LONG_PAIR', price_func=price_func, direction_func=get_direction)
    short_entry_rule = pq.PercentOfEquityTradingRule('SHORT_PAIR', price_func=price_func, long=False, direction_func=get_direction) 

    strat_builder.add_series_rule('enter_long', long_entry_rule, position_filter='zero')
    strat_builder.add_series_rule('enter_short', short_entry_rule, position_filter='zero')
    strat_builder.add_series_rule('exit_long', pq.ClosePositionExitRule('EXIT_LONG', price_func), position_filter='positive')
    strat_builder.add_series_rule('exit_short', pq.ClosePositionExitRule('EXIT_SHORT', price_func), position_filter='negative')

    strategy = strat_builder()
    strategy.run()


running typecheck
Success: no issues found in 1 source file

running flake8
flake8 success
[2023-10-18 17:29:38.212 __call__] Trade: 391 2023-01-04T09:32 AAPL 2023-01-04 09:32:00 qty: -790 prc: 126.7 order: AAPL 2023-01-04 09:31:00 qty: -790 SHORT_PAIR OrderStatus.OPEN
[2023-10-18 17:29:38.212 __call__] Trade: 391 2023-01-04T09:32 NVDA 2023-01-04 09:32:00 qty: 687 prc: 145.44 order: NVDA 2023-01-04 09:31:00 qty: 687 SHORT_PAIR OrderStatus.OPEN
[2023-10-18 17:29:38.213 __call__] Trade: 450 2023-01-04T10:31 NVDA 2023-01-04 10:31:00 qty: -687 prc: 142.955 order: NVDA 2023-01-04 10:30:00 qty: -687 EXIT_LONG OrderStatus.OPEN
[2023-10-18 17:29:38.213 __call__] Trade: 450 2023-01-04T10:31 AAPL 2023-01-04 10:31:00 qty: 790 prc: 125.63 order: AAPL 2023-01-04 10:30:00 qty: 790 EXIT_SHORT OrderStatus.OPEN
[2023-10-18 17:29:38.214 __call__] Trade: 455 2023-01-04T10:36 AAPL 2023-01-04 10:36:00 qty: 793 prc: 126.2 order: AAPL 2023-01-04 10:35:00 qty: 793 LONG_PAIR OrderStatus.OPEN
[2023-10-18 17:29:

[2023-10-18 17:29:38.237 __call__] Trade: 1969 2023-01-10T09:50 AAPL 2023-01-10 09:50:00 qty: -771 prc: 129.715 order: AAPL 2023-01-10 09:49:00 qty: -771 EXIT_LONG OrderStatus.OPEN
[2023-10-18 17:29:38.237 __call__] Trade: 1969 2023-01-10T09:50 NVDA 2023-01-10 09:50:00 qty: 645 prc: 155.36 order: NVDA 2023-01-10 09:49:00 qty: 645 EXIT_SHORT OrderStatus.OPEN
[2023-10-18 17:29:38.238 __call__] Trade: 1970 2023-01-10T09:51 AAPL 2023-01-10 09:51:00 qty: 772 prc: 129.87 order: AAPL 2023-01-10 09:50:00 qty: 772 LONG_PAIR OrderStatus.OPEN
[2023-10-18 17:29:38.238 __call__] Trade: 1970 2023-01-10T09:51 NVDA 2023-01-10 09:51:00 qty: -645 prc: 155.54 order: NVDA 2023-01-10 09:50:00 qty: -645 LONG_PAIR OrderStatus.OPEN
[2023-10-18 17:29:38.239 __call__] Trade: 2155 2023-01-10T12:56 AAPL 2023-01-10 12:56:00 qty: -772 prc: 129.72 order: AAPL 2023-01-10 12:55:00 qty: -772 EXIT_LONG OrderStatus.OPEN
[2023-10-18 17:29:38.240 __call__] Trade: 2155 2023-01-10T12:56 NVDA 2023-01-10 12:56:00 qty: 645 prc:

[2023-10-18 17:29:38.262 __call__] Trade: 2778 2023-01-12T10:19 AAPL 2023-01-12 10:19:00 qty: -749 prc: 133.59 order: AAPL 2023-01-12 10:18:00 qty: -749 SHORT_PAIR OrderStatus.OPEN
[2023-10-18 17:29:38.263 __call__] Trade: 2778 2023-01-12T10:19 NVDA 2023-01-12 10:19:00 qty: 625 prc: 160.784 order: NVDA 2023-01-12 10:18:00 qty: 625 SHORT_PAIR OrderStatus.OPEN
[2023-10-18 17:29:38.269 __call__] Trade: 3905 2023-01-18T09:36 NVDA 2023-01-18 09:36:00 qty: -625 prc: 177.13 order: NVDA 2023-01-18 09:35:00 qty: -625 EXIT_LONG OrderStatus.OPEN
[2023-10-18 17:29:38.270 __call__] Trade: 3905 2023-01-18T09:36 AAPL 2023-01-18 09:36:00 qty: 749 prc: 137.31 order: AAPL 2023-01-18 09:35:00 qty: 749 EXIT_SHORT OrderStatus.OPEN
[2023-10-18 17:29:38.271 __call__] Trade: 3910 2023-01-18T09:41 AAPL 2023-01-18 09:41:00 qty: 733 prc: 137.35 order: AAPL 2023-01-18 09:40:00 qty: 733 LONG_PAIR OrderStatus.OPEN
[2023-10-18 17:29:38.272 __call__] Trade: 3910 2023-01-18T09:41 NVDA 2023-01-18 09:41:00 qty: -571 prc

[2023-10-18 17:29:38.296 __call__] Trade: 4619 2023-01-19T15:00 AAPL 2023-01-19 15:00:00 qty: -736 prc: 136.04 order: AAPL 2023-01-19 14:59:00 qty: -736 EXIT_LONG OrderStatus.OPEN
[2023-10-18 17:29:38.297 __call__] Trade: 4619 2023-01-19T15:00 NVDA 2023-01-19 15:00:00 qty: 590 prc: 169.75 order: NVDA 2023-01-19 14:59:00 qty: 590 EXIT_SHORT OrderStatus.OPEN
[2023-10-18 17:29:38.298 __call__] Trade: 4654 2023-01-19T15:35 AAPL 2023-01-19 15:35:00 qty: 737 prc: 135.68 order: AAPL 2023-01-19 15:34:00 qty: 737 LONG_PAIR OrderStatus.OPEN
[2023-10-18 17:29:38.298 __call__] Trade: 4654 2023-01-19T15:35 NVDA 2023-01-19 15:35:00 qty: -591 prc: 169.13 order: NVDA 2023-01-19 15:34:00 qty: -591 LONG_PAIR OrderStatus.OPEN
[2023-10-18 17:29:38.299 __call__] Trade: 4656 2023-01-19T15:37 AAPL 2023-01-19 15:37:00 qty: -737 prc: 135.728 order: AAPL 2023-01-19 15:36:00 qty: -737 EXIT_LONG OrderStatus.OPEN
[2023-10-18 17:29:38.299 __call__] Trade: 4656 2023-01-19T15:37 NVDA 2023-01-19 15:37:00 qty: 591 prc:

[2023-10-18 17:29:38.328 __call__] Trade: 6636 2023-01-27T09:37 AAPL 2023-01-27 09:37:00 qty: 687 prc: 144.01 order: AAPL 2023-01-27 09:36:00 qty: 687 LONG_PAIR OrderStatus.OPEN
[2023-10-18 17:29:38.328 __call__] Trade: 6636 2023-01-27T09:37 NVDA 2023-01-27 09:37:00 qty: -506 prc: 195.347 order: NVDA 2023-01-27 09:36:00 qty: -506 LONG_PAIR OrderStatus.OPEN
[2023-10-18 17:29:38.329 __call__] Trade: 6640 2023-01-27T09:41 AAPL 2023-01-27 09:41:00 qty: -687 prc: 144.235 order: AAPL 2023-01-27 09:40:00 qty: -687 EXIT_LONG OrderStatus.OPEN
[2023-10-18 17:29:38.329 __call__] Trade: 6640 2023-01-27T09:41 NVDA 2023-01-27 09:41:00 qty: 506 prc: 197 order: NVDA 2023-01-27 09:40:00 qty: 506 EXIT_SHORT OrderStatus.OPEN
[2023-10-18 17:29:38.330 __call__] Trade: 6647 2023-01-27T09:48 AAPL 2023-01-27 09:48:00 qty: -685 prc: 144.81 order: AAPL 2023-01-27 09:47:00 qty: -685 SHORT_PAIR OrderStatus.OPEN
[2023-10-18 17:29:38.330 __call__] Trade: 6647 2023-01-27T09:48 NVDA 2023-01-27 09:48:00 qty: 498 prc: 

[2023-10-18 17:29:38.353 __call__] Trade: 7271 2023-01-30T13:42 AAPL 2023-01-30 13:42:00 qty: -692 prc: 143.025 order: AAPL 2023-01-30 13:41:00 qty: -692 EXIT_LONG OrderStatus.OPEN
[2023-10-18 17:29:38.353 __call__] Trade: 7271 2023-01-30T13:42 NVDA 2023-01-30 13:42:00 qty: 509 prc: 194.28 order: NVDA 2023-01-30 13:41:00 qty: 509 EXIT_SHORT OrderStatus.OPEN
[2023-10-18 17:29:38.354 __call__] Trade: 7274 2023-01-30T13:45 AAPL 2023-01-30 13:45:00 qty: 692 prc: 143.21 order: AAPL 2023-01-30 13:44:00 qty: 692 LONG_PAIR OrderStatus.OPEN
[2023-10-18 17:29:38.354 __call__] Trade: 7274 2023-01-30T13:45 NVDA 2023-01-30 13:45:00 qty: -509 prc: 194.1 order: NVDA 2023-01-30 13:44:00 qty: -509 LONG_PAIR OrderStatus.OPEN
[2023-10-18 17:29:38.355 __call__] Trade: 7304 2023-01-30T14:15 AAPL 2023-01-30 14:15:00 qty: -692 prc: 143.295 order: AAPL 2023-01-30 14:14:00 qty: -692 EXIT_LONG OrderStatus.OPEN
[2023-10-18 17:29:38.355 __call__] Trade: 7304 2023-01-30T14:15 NVDA 2023-01-30 14:15:00 qty: 509 prc:

Lets run the strategy, plot the results and look at the returns

In [9]:
strategy.df_roundtrip_trades()

,symbol,multiplier,entry_timestamp,exit_timestamp,qty,entry_price,exit_price,entry_reason,exit_reason,entry_commission,exit_commission,net_pnl
0,AAPL,1,2023-01-04 09:32:00,2023-01-04 10:31:00,-790,126.7,125.63,SHORT_PAIR,EXIT_SHORT,0,0,845.3
1,NVDA,1,2023-01-04 09:32:00,2023-01-04 10:31:00,687,145.43999,142.955,SHORT_PAIR,EXIT_LONG,0,0,-1707.18813
2,AAPL,1,2023-01-04 10:36:00,2023-01-04 10:47:00,793,126.2,126.822,LONG_PAIR,EXIT_LONG,0,0,493.246
3,NVDA,1,2023-01-04 10:36:00,2023-01-04 10:47:00,-698,143.2772,144.0265,LONG_PAIR,EXIT_SHORT,0,0,-523.0114
4,AAPL,1,2023-01-04 10:48:00,2023-01-04 11:03:00,787,126.82,126.7,LONG_PAIR,EXIT_LONG,0,0,-94.44
5,NVDA,1,2023-01-04 10:48:00,2023-01-04 11:03:00,-693,144.01,143.95,LONG_PAIR,EXIT_SHORT,0,0,41.58
6,AAPL,1,2023-01-04 11:32:00,2023-01-05 09:38:00,-781,127.83,125.74,SHORT_PAIR,EXIT_SHORT,0,0,1632.29
7,NVDA,1,2023-01-04 11:32:00,2023-01-05 09:38:00,679,147.12001,143.62001,SHORT_PAIR,EXIT_LONG,0,0,-2376.5
8,AAPL,1,2023-01-05 09:39:00,2023-01-05 09:40:00,793,125.7499,125.41,LONG_PAIR,EXIT_LONG,0,0,-269.5407
9,NVDA,1,2023-01-05 09:39:00,2023-01-05 09:40:00,-695,144.0607,143.16,LONG_PAIR,EXIT_SHORT,0,0,625.9865


In [8]:
strategy.evaluate_returns();

,gmean,amean,std,shrp,srt,k,calmar,mar,mdd_pct,mdd_dates,dd_3y_pct,dd_3y_timestamps,up_dwn,2023
,-0.1036,-0.1085,0.00246,-2.779,-4.273,5.822,-6.725,-6.725,0.01614,2023-01-17/2023-01-25,0.01614,2023-01-17/2023-01-25,8/10/0.444,-0.1036


2023-01-17T00:00 2023-01-25T00:00
